<a href="https://colab.research.google.com/github/mar-i0/AI-Notebooks/blob/main/Ind3xing_c0d3_by_Sha0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!export LC_ALL=en_US.UTF-8
!pip install torch
!pip install langchain
!pip install llama_index
!pip install git+https://github.com/huggingface/transformers.git accelerate
!pip install typing
!pip install PyPDF2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.5/626.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
!wget http://labs.badchecksum.net/one_function.c
!rm -rf mw/
!mkdir mw
!mv one_function.c mw/

In [6]:
import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any


# define prompt helper
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


class CustomLLM(LLM):
    #model_name = "facebook/opt-1.3b"
    model_name = "lxe/Cerebras-GPT-1.3B-Alpaca-SP"
    pipeline = pipeline("text-generation", model=model_name, device="cuda:0", model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

# define our LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

# Load the your data
print('indexing the code')
documents = SimpleDirectoryReader('./mw').load_data()
index = GPTListIndex.from_documents(documents, service_context=service_context)

response = index.query("Quién era Sancho Panza?")
print(response.response)

indexing the code


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 47>:47                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/llama_index/indices/base.py:260 in query                  │
│                                                                                                  │
│   257 │   │   │   recursive=False,                                                               │
│   258 │   │   │   use_async=use_async,                                                           │
│   259 │   │   )                                                                                  │
│ ❱ 260 │   │   return query_runner.query(query_str)                                               │
│   261 │                                                                                          │
│   262 │   async def aquery(                                                                      │
│   263 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/llama_index/indices/query/query_runner.py:349 in query    │
│                                                                                                  │
│   346 │   │   query_combiner, query_bundle = self._prepare_query_objects(                        │
│   347 │   │   │   query_str_or_bundle, index_id=index_id                                         │
│   348 │   │   )                                                                                  │
│ ❱ 349 │   │   return query_combiner.run(query_bundle, level)                                     │
│   350 │                                                                                          │
│   351 │   async def aquery(                                                                      │
│   352 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/llama_index/indices/query/query_combiner/base.py:68 in    │
│ run                                                                                              │
│                                                                                                  │
│    65 │   def run(self, query_bundle: QueryBundle, level: int = 0) -> RESPONSE_TYPE:             │
│    66 │   │   """Run query combiner."""                                                          │
│    67 │   │   updated_query_bundle = self._prepare_update(query_bundle)                          │
│ ❱  68 │   │   return self._query_runner.query_transformed(                                       │
│    69 │   │   │   updated_query_bundle, self._index_struct, level=level                          │
│    70 │   │   )                                                                                  │
│    71                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/llama_index/indices/query/query_runner.py:209 in          │
│ query_transformed                                                                                │
│                                                                                                  │
│   206 │   │   │   )                                                                              │
│   207 │   │   │   return response                                                                │
│   208 │   │   else:                                        

In [ ]:
response = index.query("what is doing the function some_function?")
print(response.response)

NameError: ignored